In [4]:
# Importation des librairies
import streamlit as st
from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, date
import plotly.express as px
import plotly.tools as tls
from plotly.offline import iplot

In [20]:
import pandas as pd

# URLs pour les indices boursiers
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # USA
url_cac40 = "https://en.wikipedia.org/wiki/CAC_40"  # Paris
url_ftse100 = "https://en.wikipedia.org/wiki/FTSE_100_Index"  # Londres
url_nikkei = "https://topforeignstocks.com/indices/the-components-of-the-nikkei-225-index/"  # Tokyo
url_dax = "https://en.wikipedia.org/wiki/DAX"  # Berlin

# Fonction pour lire les tables en toute sécurité
def read_table(url, index):
    try:
        df = pd.read_html(url)[index]
        print(f"Colonnes dans la table de {url} : {df.columns.tolist()}")  # Inspecter les colonnes
        return df
    except Exception as e:
        print(f"Erreur lors de la lecture de la table à partir de {url} : {e}")
        return pd.DataFrame()  # Retourne un DataFrame vide en cas d'erreur

# Lire et traiter le DAX
# (l'indice boursier des 30 plus grandes entreprises cotées en bourse en Allemagne)
dax = read_table(url_dax, 3)
if 'Company' in dax.columns and 'Ticker symbol' in dax.columns:
    dax['NameOfStock'] = dax['Company'] + "_" + dax['Ticker symbol']
else:
    print("La table DAX ne contient pas les colonnes attendues.")

# Lire et traiter le Nikkei
# (l'indice boursier le plus suivi au Japon et comprend 225 grandes entreprises japonaises)
nikkei = read_table(url_nikkei, 0)
if 'Company Name' in nikkei.columns and 'Code' in nikkei.columns:
    nikkei['Company Name'] = nikkei['Company Name'].replace(",", "", regex=True)  # Supprimer les virgules
    nikkei['NameOfStock'] = nikkei['Company Name'] + "_" + nikkei['Code'].astype(str) + ".T"
else:
    print("La table Nikkei ne contient pas les colonnes attendues.")

# Lire et traiter le S&P 500
# (l'indice boursier comprenant 500 des plus grandes entreprises cotées en bourse aux États-Unis)
sp500 = read_table(url_sp500, 0)
if 'Security' in sp500.columns and 'Symbol' in sp500.columns:
    sp500['NameOfStock'] = sp500['Security'] + "_" + sp500['Symbol']
else:
    print("La table S&P 500 ne contient pas les colonnes attendues.")

# Lire et traiter le CAC 40
# l'indice boursier des 40 plus grandes entreprises cotées en France.
cac40 = read_table(url_cac40, 3)
if 'Company' in cac40.columns and 'Ticker' in cac40.columns:
    cac40['NameOfStock'] = cac40['Company'] + "_" + cac40['Ticker']
else:
    print("La table CAC 40 ne contient pas les colonnes attendues.")

# Lire et traiter le FTSE 100
# (l'indice boursier des 100 plus grandes entreprises cotées à la Bourse de Londres).
ftse100 = read_table(url_ftse100, 3)
if 'Company' in ftse100.columns and 'EPIC' in ftse100.columns:
    ftse100['NameOfStock'] = ftse100['Company'] + "_" + ftse100["EPIC"]
else:
    print("La table FTSE 100 ne contient pas les colonnes attendues.")

# Optionnellement, combiner tous les DataFrames en un seul
combined_df = pd.concat([dax, nikkei, sp500, cac40, ftse100], ignore_index=True)

# Afficher le DataFrame combiné
print(combined_df.head())

Colonnes dans la table de https://en.wikipedia.org/wiki/DAX : ['Year', 'Closing level', 'Change in Index in Points', 'Change in Index in\xa0%']
La table DAX ne contient pas les colonnes attendues.
Erreur lors de la lecture de la table à partir de https://topforeignstocks.com/indices/the-components-of-the-nikkei-225-index/ : HTTP Error 403: Forbidden
La table Nikkei ne contient pas les colonnes attendues.
Colonnes dans la table de https://en.wikipedia.org/wiki/List_of_S%26P_500_companies : ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']
Colonnes dans la table de https://en.wikipedia.org/wiki/CAC_40 : ['Year', 'Closing level', 'Change in Index in Points', 'Change in Index in\xa0%']
La table CAC 40 ne contient pas les colonnes attendues.
Colonnes dans la table de https://en.wikipedia.org/wiki/FTSE_100_Index : [('Year', 'Year'), ('Closing level', 'Closing level'), ('Change in index', '(points)'), ('Change in index', '(%)')

In [26]:
# load_data permet de récupérer des données historiques pour des actions spécifiques
def load_data(symbol, start_date, end_date):
	stock_data = DataReader(symbol, data_source = "yahoo", start = start_date, end = end_date)
	return stock_data
# facilite l'accès aux noms des actions disponibles sur différents indices boursiers. 
def list_of_stocks(market_name):
	if market_name == "SP500 (USA)":
		stocks = sp500['NameOfStock'].to_list()
	elif market_name == "CAC 40 (France)":
		stocks = cac40['NameOfStock'].to_list()
	elif market_name == "FTSE 100 (Angleterre)":
		stocks = ftse100['NameOfStock'].to_list()
	elif market_name == "NIKKEI (Japon)":
		stocks = nikkei['NameOfStock'].to_list()
	else:
		stocks = dax['NameofStock'].to_list()
	return stocks

# Ensemble, elles simplifient le processus d'analyse et de visualisation des données financières.

In [34]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.tools as tls
from pandas_datareader.data import DataReader
from datetime import date

# Fonction pour charger les données de prix d'une action
def load_data(symbol, start_date, end_date):
    try:
        stock_data = DataReader(symbol, data_source="yahoo", start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        st.error(f"Erreur lors du chargement des données : {e}")
        return None

# Fonction pour obtenir la liste des actions d'un marché spécifique
def list_of_stocks(market_name):
    if market_name == "SP500 (USA)":
        return sp500['NameOfStock'].to_list()
    elif market_name == "CAC 40 (France)":
        return cac40['NameOfStock'].to_list()
    elif market_name == "FTSE 100 (Angleterre)":
        return ftse100['NameOfStock'].to_list()
    elif market_name == "NIKKEI (Japon)":
        return nikkei['NameOfStock'].to_list()
    else:
        return dax['NameofStock'].to_list()

# Interface utilisateur
st.title("📈 Analyse des Marchés Boursiers")
st.sidebar.header("Configuration de l'Analyse")

# Choix du marché boursier
market = st.sidebar.selectbox('Choisissez un Marché Boursier', ["SP500 (USA)", "CAC 40 (France)", "FTSE 100 (Angleterre)", "NIKKEI (Japon)", "DAX (Allemagne)"])

# Récupération de la liste des actions pour le marché choisi
stocks_list = list_of_stocks(market_name=market)

# Choix d'une action à analyser
stock = st.sidebar.selectbox('Choisissez une Action à Analyser', stocks_list)

# Choix de la période d'analyse
st.sidebar.write('Choisissez une Période d\'Analyse :')
date1 = st.sidebar.date_input("Date de Début", value=date(2017, 1, 1))
date2 = st.sidebar.date_input("Date de Fin", value=date(2024, 1, 1))

# Choix des moyennes mobiles
st.sidebar.header("Paramètres des Moyennes Mobiles")
short = st.sidebar.slider("Moyenne Mobile Courte (jours)", min_value=0, max_value=200, value=20)
long = st.sidebar.slider("Moyenne Mobile Longue (jours)", min_value=0, max_value=200, value=100)

# Récupération des données boursières
df = load_data(symbol=stock.split("_")[1], start_date=date1, end_date=date2)

if df is not None:
    # Affichage des données brutes si l'option est cochée
    if st.sidebar.checkbox("Afficher les Données Brutes", False):
        st.subheader(f"Données de {stock.split('_')[0]}")
        st.write(df)

    # Création des graphiques
    st.header(f"Analyse de {stock.split('_')[1]}")
    
    # Boxplot des prix
    box = px.box(df, y="Close", title=f"{stock.split('_')[1]} - Prix de Clôture", color_discrete_sequence=['#1f77b4'])
    st.plotly_chart(box, use_container_width=True)

    # Graphique du volume
    volume_chart = px.line(df.reset_index(), x="Date", y="Volume", title=f"{stock.split('_')[1]} - Volume", color_discrete_sequence=['#ff7f0e'])
    st.plotly_chart(volume_chart, use_container_width=True)

    # Calcul des moyennes mobiles
    short_rolling = df['Close'].rolling(window=short).mean()
    long_rolling = df['Close'].rolling(window=long).mean()

    # Calcul des signaux d'achat et de vente
    df['Signal'] = 0.0
    df['Signal'] = np.where(short_rolling > long_rolling, 1.0, 0.0)
    df['Position'] = df['Signal'].diff()

    # Tracé des données avec matplotlib
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(df['Close'].index, df['Close'], label=f'Prix de Clôture', color='blue')
    ax.plot(short_rolling.index, short_rolling, label=f'{short} jours Rolling', color='orange')
    ax.plot(long_rolling.index, long_rolling, label=f'{long} jours Rolling', color='green')

    # Tracé des signaux d'achat et de vente
    ax.plot(df[df['Position'] == 1].index, short_rolling[df['Position'] == 1], '^', markersize=15, color='black', label='Acheter')
    ax.plot(df[df['Position'] == -1].index, short_rolling[df['Position'] == -1], 'v', markersize=15, color='red', label='Vendre')
    
    # Configuration des axes et légende
    ax.set_xlabel('Date')
    ax.set_ylabel('Prix de Clôture ($)')
    ax.set_title(f'Analyse des Moyennes Mobiles pour {stock.split("_")[1]}')
    ax.legend()

    # Conversion et affichage avec Plotly
    plotly_fig = tls.mpl_to_plotly(fig)
    st.plotly_chart(plotly_fig, use_container_width=True)
else:
    st.warning("Aucune donnée disponible pour l'action sélectionnée.")
